In [2]:
import numpy as np
import matplotlib.pylab as plt
from tempfile import NamedTemporaryFile
from IPython.display import HTML
from matplotlib import animation

x = np.linspace(0,1,51)

# Smooth Rectangle
f = 1. / (1. + np.exp(80.*(np.abs(x-0.5) - 0.15)))

# Triangle
#f = np.zeros((51))
#for i in range(13,26):
#    f[i] = (4.)*(x[i]-0.25)
#for i in range(26,38):
#    f[i] = (-4.)*(x[i]-0.75)


exact_f = np.empty((251,51))
f2 = f
exact_f[0,:] = f2
for i in range(1,251):
    if (i%5 == 0):
        f2 = np.roll(f2,2)
    if ((i+50)%125 == 0):
        f2 = np.roll(f2,1)
    exact_f[i,:] = f2

In [3]:
# Sixth Order Spatial, Leapfrog in time
cnum = 0.4
gamma = 0.025

qc6 = np.empty((251,51))
qf6 = np.empty((251,51))
qc6[0,:] = f
qf6[0,:] = f

# First time step, upwind
qc6[1,0] = qc6[0,0] - cnum*(qc6[0,1]-qc6[0,49])
for j in range(1,50):
    qc6[1,j] = qc6[0,j] - cnum*(qc6[0,j+1]-qc6[0,j-1])
qc6[1,50] = qc6[1,0]


for i in range(2,251):
    qc6[i,0] = qf6[i-2,0] - cnum*2.*(45.*(qc6[i-1,1]-qc6[i-1,49]) + 9.*(qc6[i-1,48] - qc6[i-1,2]) 
                                   + qc6[i-1,3] - qc6[i-1,47])/60.
    qc6[i,1] = qf6[i-2,1] - cnum*2.*(45.*(qc6[i-1,2]-qc6[i-1,0]) + 9.*(qc6[i-1,49] - qc6[i-1,3]) 
                                   + qc6[i-1,4] - qc6[i-1,48])/60.
    qc6[i,2] = qf6[i-2,2] - cnum*2.*(45.*(qc6[i-1,3]-qc6[i-1,1]) + 9.*(qc6[i-1,0] - qc6[i-1,4]) 
                                   + qc6[i-1,5] - qc6[i-1,49])/60.
    
    for j in range(3,48):
        qc6[i,j] = qf6[i-2,j] - cnum*2.*(45.*(qc6[i-1,j+1]-qc6[i-1,j-1]) + 9.*(qc6[i-1,j-2] - qc6[i-1,j+2]) 
                                       + qc6[i-1,j+3] - qc6[i-1,j-3])/60.
    # BCs
    qc6[i,48] = qf6[i-2,48] - cnum*2.*(45.*(qc6[i-1,49]-qc6[i-1,47]) + 9.*(qc6[i-1,46] - qc6[i-1,0])
                                     + qc6[i-1,1] - qc6[i-1,45])/60.
    qc6[i,49] = qf6[i-2,49] - cnum*2.*(45.*(qc6[i-1,0]-qc6[i-1,48]) + 9.*(qc6[i-1,47] - qc6[i-1,1])
                                     + qc6[i-1,1] - qc6[i-1,46])/60.
    qc6[i,50] = qc6[i,0]
    
    # Asselin Filter to control computational mode with Leapfrog in time
    for j in range(51):
        qf6[i-1,j] = qc6[i-1,j] + gamma*(qc6[i,j]-2*qc6[i-1,j]+qf6[i-2,j])
leap6 = qc6
TRER = np.sqrt(np.sum((f - leap6[250,:])**2./50.))

In [4]:
# Sixth Order Spatial, RK3
cnum = 0.4
gamma = 0.025

qc6 = np.empty((251,51))
rk3_1 = np.empty((251,51))
rk3_2 = np.empty((251,51))
qc6[0,:] = f
rk3_1[0,:] = 0
rk3_2[0,:] = 0


for i in range(1,251):
    # First RK3 Step
    rk3_1[i,0] = qc6[i-1,0] - cnum/3.*(45.*(qc6[i-1,1]-qc6[i-1,49]) + 9.*(qc6[i-1,48] - qc6[i-1,2]) 
                                   + qc6[i-1,3] - qc6[i-1,47])/60.
    rk3_1[i,1] = qc6[i-1,1] - cnum/3.*(45.*(qc6[i-1,2]-qc6[i-1,0]) + 9.*(qc6[i-1,49] - qc6[i-1,3]) 
                                   + qc6[i-1,4] - qc6[i-1,48])/60.
    rk3_1[i,2] = qc6[i-1,2] - cnum/3.*(45.*(qc6[i-1,3]-qc6[i-1,1]) + 9.*(qc6[i-1,0] - qc6[i-1,4]) 
                                   + qc6[i-1,5] - qc6[i-1,49])/60.
    
    for j in range(3,48):
        rk3_1[i,j] = qc6[i-1,j] - cnum/3.*(45.*(qc6[i-1,j+1]-qc6[i-1,j-1]) + 9.*(qc6[i-1,j-2] - qc6[i-1,j+2]) 
                                       + qc6[i-1,j+3] - qc6[i-1,j-3])/60.
    # BCs
    rk3_1[i,48] = qc6[i-1,48] - cnum/3.*(45.*(qc6[i-1,49]-qc6[i-1,47]) + 9.*(qc6[i-1,46] - qc6[i-1,0])
                                     + qc6[i-1,1] - qc6[i-1,45])/60.
    rk3_1[i,49] = qc6[i-1,49] - cnum/3.*(45.*(qc6[i-1,0]-qc6[i-1,48]) + 9.*(qc6[i-1,47] - qc6[i-1,1])
                                     + qc6[i-1,2] - qc6[i-1,46])/60.
    rk3_1[i,50] = rk3_1[i,0]
    
    
    # Second RK3 Step
    rk3_2[i,0] = qc6[i-1,0] - cnum/2.*(45.*(rk3_1[i,1]-rk3_1[i,49]) + 9.*(rk3_1[i,48] - rk3_1[i,2]) 
                                   + rk3_1[i,3] - rk3_1[i,47])/60.
    rk3_2[i,1] = qc6[i-1,1] - cnum/2.*(45.*(rk3_1[i,2]-rk3_1[i,0]) + 9.*(rk3_1[i,49] - rk3_1[i,3]) 
                                   + rk3_1[i,4] - rk3_1[i,48])/60.
    rk3_2[i,2] = qc6[i-1,2] - cnum/2.*(45.*(rk3_1[i,3]-rk3_1[i,1]) + 9.*(rk3_1[i,0] - rk3_1[i,4]) 
                                   + rk3_1[i,5] - rk3_1[i,49])/60.
    
    for j in range(3,48):
        rk3_2[i,j] = qc6[i-1,j] - cnum/2.*(45.*(rk3_1[i,j+1]-rk3_1[i,j-1]) + 9.*(rk3_1[i,j-2] - rk3_1[i,j+2]) 
                                       + rk3_1[i,j+3] - rk3_1[i,j-3])/60.
    # BCs
    rk3_2[i,48] = qc6[i-1,48] - cnum/2.*(45.*(rk3_1[i,49]-rk3_1[i,47]) + 9.*(rk3_1[i,46] - rk3_1[i,0])
                                     + rk3_1[i,1] - rk3_1[i,45])/60.
    rk3_2[i,49] = qc6[i-1,49] - cnum/2.*(45.*(rk3_1[i,0]-rk3_1[i,48]) + 9.*(rk3_1[i,47] - rk3_1[i,1])
                                     + rk3_1[i,2] - rk3_1[i,46])/60.
    rk3_2[i,50] = rk3_2[i,0]
    
    
    # Third RK3 Step
    qc6[i,0] = qc6[i-1,0] - cnum*(45.*(rk3_2[i,1]-rk3_2[i,49]) + 9.*(rk3_2[i,48] - rk3_2[i,2]) 
                                   + rk3_2[i,3] - rk3_2[i,47])/60.
    qc6[i,1] = qc6[i-1,1] - cnum*(45.*(rk3_2[i,2]-rk3_2[i,0]) + 9.*(rk3_2[i,49] - rk3_2[i,3]) 
                                   + rk3_2[i,4] - rk3_2[i,48])/60.
    qc6[i,2] = qc6[i-1,2] - cnum*(45.*(rk3_2[i,3]-rk3_2[i,1]) + 9.*(rk3_2[i,0] - rk3_2[i,4]) 
                                   + rk3_2[i,5] - rk3_2[i,49])/60.
    
    for j in range(3,48):
        qc6[i,j] = qc6[i-1,j] - cnum*(45.*(rk3_2[i,j+1]-rk3_2[i,j-1]) + 9.*(rk3_2[i,j-2] - rk3_2[i,j+2]) 
                                       + rk3_2[i,j+3] - rk3_2[i,j-3])/60.
    # BCs
    qc6[i,48] = qc6[i-1,48] - cnum*(45.*(rk3_2[i,49]-rk3_2[i,47]) + 9.*(rk3_2[i,46] - rk3_2[i,0])
                                     + rk3_2[i,1] - rk3_2[i,45])/60.
    qc6[i,49] = qc6[i-1,49] - cnum*(45.*(rk3_2[i,0]-rk3_2[i,48]) + 9.*(rk3_2[i,47] - rk3_2[i,1])
                                     + rk3_2[i,2] - rk3_2[i,46])/60.
    qc6[i,50] = qc6[i,0]
rk6 = qc6
TRER = np.sqrt(np.sum((f - rk6[250,:])**2./50.))

In [5]:
%matplotlib inline
VIDEO_TAG = """<video controls>
 <source src="data:video/x-m4v;base64,{0}" type="video/mp4">
 Your browser does not support the video tag.
</video>"""

def anim_to_html(anim):
    if not hasattr(anim, '_encoded_video'):
        with NamedTemporaryFile(suffix='.mp4') as f:
            anim.save(f.name, fps=20,dpi=150,extra_args=['-vcodec', 'libx264'])
            video = open(f.name, "rb").read()
        anim._encoded_video = video.encode("base64")
    
    return VIDEO_TAG.format(anim._encoded_video)

def display_animation(anim):
    plt.close(anim._fig)
    return HTML(anim_to_html(anim))

# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure()
ax = plt.axes(xlim=(0, 1), ylim=(-0.5, 1.5))
plt.title('6th Order in Spatial Diff')
line, = ax.plot([], [], lw=1, label='Leapfrog')
line3, = ax.plot([], [], lw=1,color='darkcyan', label='RK3')
line2, = ax.plot([], [], lw=1,ls='dashed',color='r')
time_template = 't = %.0f'
time_text = ax.text(0.05, 1.4, '')
ax.axvline(0.5,ls='dotted',color='k')
ax.legend(handles=[line,line3],loc='lower center',fontsize=8,ncol=2)
x2 = x

# initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    line2.set_data([], [])
    line3.set_data([], [])
    time_text.set_text('')
    return line, line2, line3, time_text

# animation function.  This is called sequentially
def animate(i):
    #x = np.linspace(0, 2, 1000)
    #y = np.sin(2 * np.pi * (x - 0.01 * i))
    line.set_data(x, leap6[i,:])
    line3.set_data(x, rk6[i,:])
    line2.set_data(x, exact_f[i,:])
    time_text.set_text(time_template % (i))
    return line, line2, line3, time_text

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=251, interval=20, blit=True)

# call our new function to display the animation
display_animation(anim)

# To same animation, comment out previous line and
# uncomment the next two lines.
#fname = 'rk3_time_4th_order_space_box.mp4'
#anim.save(fname, fps=20,dpi=500)